In [1]:

!pip install faker streamlit

In [2]:

import sqlite3
from faker import Faker
import random
import pandas as pd
from datetime import datetime

In [3]:

class DatabaseManager:
    def __init__(self, db_name="students.db"):
        self.db_name = db_name

    def create_tables(self):
        conn = sqlite3.connect(self.db_name)
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS students (
                student_id INTEGER PRIMARY KEY,
                name TEXT, age INTEGER, email TEXT, phone TEXT, enrollment_date TEXT
            );
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS programming (
                student_id INTEGER, language TEXT,
                problems_solved INTEGER, assessments_completed INTEGER, projects_submitted INTEGER,
                FOREIGN KEY (student_id) REFERENCES students(student_id)
            );
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS soft_skills (
                student_id INTEGER,
                communication_score INTEGER, teamwork_score INTEGER, presentation_score INTEGER,
                FOREIGN KEY (student_id) REFERENCES students(student_id)
            );
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS placements (
                student_id INTEGER,
                placement_ready TEXT, mock_interview_score INTEGER, internship_experience TEXT,
                FOREIGN KEY (student_id) REFERENCES students(student_id)
            );
        """)
        conn.commit()
        conn.close()

    def insert(self, table, data):
        try:
            conn = sqlite3.connect(self.db_name, check_same_thread=False)
            cursor = conn.cursor()
            placeholders = ','.join('?' * len(data[0]))
            cursor.executemany(f"INSERT INTO {table} VALUES ({placeholders})", data)
            conn.commit()
        except sqlite3.OperationalError as e:
            print("Error inserting data:", e)
        finally:
            conn.close()

In [4]:

fake = Faker()

def generate_students(n=5):
    return [
        (
            i,
            fake.name(),
            random.randint(118,135),
            fake.email(),
            fake.phone_number(),
            fake.date_between(start_date='-2y', end_date='today').isoformat()
        ) for i in range(1, n + 1)
    ]

def generate_programming(n=5):
    return [(i, 'Python', random.randint(20, 100), random.randint(1, 5), random.randint(0, 3)) for i in range(1, n+1)]

def generate_soft_skills(n=5):
    return [(i, random.randint(60, 100), random.randint(60, 100), random.randint(60, 100)) for i in range(1, n+1)]

def generate_placements(n=5):
    return [(i, random.choice(['Yes', 'No']), random.randint(40, 100), random.choice(['Yes', 'No'])) for i in range(1, n+1)]

In [5]:

db = DatabaseManager()
db.create_tables()

students = generate_students()
programming = generate_programming()
soft_skills = generate_soft_skills()
placements = generate_placements()

db.insert("students", students)
db.insert("programming", programming)
db.insert("soft_skills", soft_skills)
db.insert("placements", placements)

In [6]:

conn = sqlite3.connect("students.db")
df = pd.read_sql_query('''
SELECT s.name, p.problems_solved, 
       (ss.communication_score + ss.teamwork_score + ss.presentation_score)/3.0 AS soft_skills_avg,
       place.placement_ready
FROM students s
JOIN programming p ON s.student_id = p.student_id
JOIN soft_skills ss ON s.student_id = ss.student_id
JOIN placements place ON s.student_id = place.student_id;
''', conn)
df

,name,problems_solved,soft_skills_avg,placement_ready
0,Alan Jordan,93,76.666667,Yes
1,Frank Moore,62,83.333333,Yes
2,Megan Lopez,73,81.000000,Yes
3,Kimberly Randall,37,78.666667,Yes
4,Steven Scott,55,81.333333,Yes
